In [3]:
import pandas as pd
import random #Randomize the time
import requests
from time import sleep
from requests import get #Time delay for webscraping

from bs4 import BeautifulSoup as soup #Webscaping
import sqlite3 
from api_key import key

#Change your API Key here
api_key = key

1) SQL Database 

In [4]:
#Scrape 10 different pages to get the list of 1000 top actors
nameList=pd.DataFrame({"Rank":[],"Name":[] })
for pageindex in [1,2,3,4,5,6,7,8,9,10]:
    url="https://www.imdb.com/list/ls058011111/?sort=list_order,asc&mode=detail&page="+str(pageindex)+"&ref_=nmls_vm_dtl"
    response = get(url)
    html_soup=soup(response.text, "html.parser")
    page=html_soup.find_all(class_="lister-item-content")
    for index, element in enumerate(page):
        nameList.loc[len(nameList)]=[index+1+(pageindex-1)*100,element.find_all('a')[0].text[:-1]]
    #This function randomize the delay time to avoid overload a website
    sleep(random.random())

In [ ]:
#Create a rank List database
!rm 'actorRank.db' #Avoid duplicating the file
conn = sqlite3.connect('actorRank.db')
cur=conn.cursor()

#Create table "rank" in the "actorRank" database. This have 2 columns: Rank, Name
cur.execute("""CREATE TABLE rank(Rank Integer, Name TEXT);""")

#Insert values from the "nameList" created above to the table
for i in range(nameList.shape[0]):
    name=nameList['Name'][i].replace("'"," ")
    rank=nameList['Rank'][i]
    #Note that there is a '' around {}
    cur.execute("""INSERT INTO rank (Rank,Name) VALUES ({},'{}');""".format(rank,name))

conn.commit()

2) Movies CVS

In [5]:
#Given a year, return data frame of that year

def moviesData(year):
    urlDiscover='https://api.themoviedb.org/3/discover/movie?api_key='
    searchkey='&primary_release_year='+str(year)+'&sort_by=revenue.desc'
    response = requests.get(urlDiscover + api_key + searchkey)
    revenueResult = response.json() 
    revenueResultLimit= revenueResult['results']

    df = pd.DataFrame(columns=['Release_Date','ID','Title','Budget','Revenue','Profit','Runtime','Rating','Genres'])
    
    for film in revenueResultLimit:
        filmDetails = requests.get('https://api.themoviedb.org/3/movie/'+ str(film['id']) +'?api_key='+ api_key +'&language=en-US&append_to_response=release_dates')
        filmDetails = filmDetails.json()
        profit=filmDetails['revenue']-filmDetails['budget']
        df.loc[len(df)]=[film['release_date'],film['id'],film['title'],filmDetails['budget'],filmDetails['revenue'], profit,filmDetails['runtime'],filmDetails['vote_average'],genres(filmDetails)]
    return df

In [6]:
#This is a supplement function for the above function
#Give the json file "filmDetails", return the list of all genres
def genres(filmDetails):
    genresList=[]
    for i in range(len(filmDetails['genres'])):
        genresList.append(filmDetails['genres'][i]['name'])
    return genresList

In [7]:
#Create a list of Data Frame from 2009 to 2019
yearList=[]
for i in range(11):
    year=2009+i
    yearList.append(moviesData(year))
    sleep(6)

In [8]:
#Create a df of all 10 years
!rm moviesData
df=pd.concat(yearList,ignore_index=True)
#Save that df in a csv file
df.to_csv('moviesData')

In [10]:
df.tail()

,Release_Date,ID,Title,Budget,Revenue,Profit,Runtime,Rating,Genres
215,2019-06-14,479455,Men in Black: International,110000000,252608480,142608480,115,5.9,"[Action, Comedy, Science Fiction, Adventure]"
216,2019-01-18,450465,Glass,20000000,246941965,226941965,129,6.6,"[Thriller, Drama, Science Fiction]"
217,2019-06-07,320288,Dark Phoenix,200000000,245000000,45000000,114,6.1,"[Science Fiction, Action, Adventure]"
218,2019-09-06,474350,It Chapter Two,79000000,237168524,158168524,169,7.2,"[Horror, Comedy]"
219,2019-03-29,329996,Dumbo,170000000,215832199,45832199,112,6.6,"[Adventure, Family, Fantasy]"


In [ ]:
test = https://api.themoviedb.org/3/movie/479455/release_dates?api_key=cf92d1cf751a68bde1f3b3aa76b677be


In [2]:
def shout(phrase):
  if phrase == phrase.upper():
    return "YOU'RE SHOUTING!"
  else:
    return "Can you speak up?"

shout("HELLO")

"YOU'RE SHOUTING!"